In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel, AdamW

# Load model and tokenizer
MODEL_PATH ="/Users/sir/Downloads/HuggingFace/sentence_transformer/intfloat_e5-large-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModel.from_pretrained(MODEL_PATH)

# Example batch of paired inputs: (query, positive passage)
texts = [
    ("how much protein should a female eat", "The CDC average protein requirement for women is 46 grams per day."),
    ("define summit", "Definition of summit is the highest point of a mountain.")
]

ImportError: cannot import name 'AdamW' from 'transformers' (/opt/homebrew/Caskroom/miniconda/base/envs/LLM/lib/python3.13/site-packages/transformers/__init__.py)

In [ ]:
# Tokenize batch
queries, positives = zip(*texts)
query_inputs = tokenizer(list(queries), return_tensors='pt', padding=True, truncation=True)
positive_inputs = tokenizer(list(positives), return_tensors='pt', padding=True, truncation=True)

In [ ]:
# Forward pass to get embeddings
query_embeds = model(**query_inputs).last_hidden_state.mean(dim=1)
positive_embeds = model(**positive_inputs).last_hidden_state.mean(dim=1)

# Normalize embeddings
query_embeds = F.normalize(query_embeds, p=2, dim=1)
positive_embeds = F.normalize(positive_embeds, p=2, dim=1)

# Compute cosine similarity matrix
similarity_matrix = torch.matmul(query_embeds, positive_embeds.t())

# Create labels (diagonal matches)
labels = torch.arange(len(query_embeds)).to(similarity_matrix.device)

# Temperature parameter to scale similarities
temperature = 0.05
logits = similarity_matrix / temperature

# Contrastive loss with cross-entropy
loss = F.cross_entropy(logits, labels)

# Backprop and optimization
optimizer = AdamW(model.parameters(), lr=5e-5)
optimizer.zero_grad()
loss.backward()
optimizer.step()

print(f"Contrastive loss: {loss.item()}")